In [1]:
!pip install librecommender

     -------------------------------------- 837.5/837.5 kB 1.0 MB/s eta 0:00:00
     ---------------------------------------- 24.0/24.0 MB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 61.2/61.2 kB 3.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
weasel 0.3.4 requires smart-open<7.0.0,>=5.2.1, but you have smart-open 7.0.4 which is incompatible.
spacy 3.7.4 requires smart-open<7.0.0,>=5.2.1, but you have smart-open 7.0.4 which is incompatible.
fastai 2.7.14 requires torch<2.3,>=1.10, but you have torch 2.3.1 which is incompatible.


In [1]:
import pandas as pd
from libreco.data import DatasetPure
from libreco.algorithms import ItemCF
from libreco.evaluation import evaluate

Instructions for updating:
non-resource variables are not supported in the long term


D:\Anaconda\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
train_ratings = pd.read_csv('processed_dataset/MovieLens-1M/ratings/ratings_traindata_movielens.csv')
val_ratings = pd.read_csv('processed_dataset/MovieLens-1M/ratings/ratings_valdata_movielens.csv')
test_ratings = pd.read_csv('processed_dataset/MovieLens-1M/ratings/ratings_testdata_movielens.csv')

In [4]:
# Rename columns to match the expected format
train_ratings.rename(columns={'user_id': 'user', 'item_id': 'item', 'rating': 'label', 'timestamp': 'time'}, inplace=True)
val_ratings.rename(columns={'user_id': 'user', 'item_id': 'item', 'rating': 'label', 'timestamp': 'time'}, inplace=True)
test_ratings.rename(columns={'user_id': 'user', 'item_id': 'item', 'rating': 'label', 'timestamp': 'time'}, inplace=True)

# Ensure the columns are in the correct order
train_ratings = train_ratings[['user', 'item', 'label', 'time']]
val_ratings = val_ratings[['user', 'item', 'label', 'time']]
test_ratings = test_ratings[['user', 'item', 'label', 'time']]

In [5]:
train_data, data_info = DatasetPure.build_trainset(train_ratings)
eval_data = DatasetPure.build_evalset(val_ratings)
test_data = DatasetPure.build_testset(test_ratings)
print(data_info)  # n_users: 5894, n_items: 3253, data sparsity: 0.4172 %

n_users: 6040, n_items: 3667, data density: 3.6018 %


In [7]:
item_cf = ItemCF(task="ranking", data_info=data_info, k_sim=100, sim_type="cosine", mode='invert')

In [8]:
# Training the model
item_cf.fit(train_data, verbose=2, eval_data=eval_data, k=5, metrics=["loss", "roc_auc", "precision", "recall", "ndcg"], neg_sampling=True)

Training start time: 2024-07-27 09:02:02
Final block size and num: (3667, 1)
sim_matrix elapsed: 1.083s
sim_matrix, shape: (3667, 3667), num_elements: 8846950, density: 65.7918 %


eval_pointwise:   0%|          | 0/25 [00:00<?, ?it/s]

Detect 1 unknown interaction(s), position: [5602]
No common interaction or similar neighbor for user 0 and item 699, proceed with default prediction
No common interaction or similar neighbor for user 0 and item 2344, proceed with default prediction
No common interaction or similar neighbor for user 0 and item 577, proceed with default prediction
No common interaction or similar neighbor for user 0 and item 91, proceed with default prediction
No common interaction or similar neighbor for user 0 and item 570, proceed with default prediction
No common interaction or similar neighbor for user 0 and item 1008, proceed with default prediction


eval_pointwise:   4%|▍         | 1/25 [00:00<00:10,  2.20it/s]

Detect 1 unknown interaction(s), position: [5320]


...

Detect 2 unknown interaction(s), position: [1280, 3058]


eval_pointwise:  84%|████████▍ | 21/25 [00:10<00:01,  2.05it/s]

Detect 1 unknown interaction(s), position: [3300]


eval_listwise: 100%|██████████| 2997/2997 [02:01<00:00, 24.70it/s]

	 eval log_loss: 1.8914
	 eval roc_auc: 0.7569
	 eval precision@5: 0.0765
	 eval recall@5: 0.0430
	 eval ndcg@5: 0.1815


In [9]:
# Evaluate the model on the test data with the specified metrics
evaluation_results = evaluate(
    model=item_cf,
    data=test_data,
    neg_sampling=True,
    metrics=["loss", "roc_auc", "precision", "recall", "ndcg"]
)

# Print the evaluation results
for metric, value in evaluation_results.items():
    print(f"{metric}: {value}")

eval_pointwise:   0%|          | 0/26 [00:00<?, ?it/s]

Detect 1 unknown interaction(s), position: [1344]


eval_pointwise:   8%|▊         | 2/26 [00:00<00:11,  2.17it/s]

Detect 2 unknown interaction(s), position: [1890, 812]


eval_pointwise:  19%|█▉        | 5/26 [00:02<00:10,  2.04it/s]

...

Detect 2 unknown interaction(s), position: [4738, 4766]


eval_pointwise:  92%|█████████▏| 24/26 [00:11<00:00,  2.15it/s]

Detect 1 unknown interaction(s), position: [8184]


eval_listwise: 100%|██████████| 3000/3000 [02:02<00:00, 24.40it/s]


loss: 1.9961144138330738
roc_auc: 0.729017611866037
precision: 0.05485
recall: 0.056465052861735636
ndcg: 0.1849518364464186


In [10]:
import numpy as np

def dcg_at_k(y_true_sorted, k):
    y_true_sorted = np.asarray(y_true_sorted)[:k]
    return np.sum((2**y_true_sorted - 1) / np.log2(np.arange(2, len(y_true_sorted) + 2)))

def ndcg_at_k(y_true, y_score, k):
    order = np.argsort(y_score)[::-1]
    y_true_sorted = np.take(y_true, order[:k])
    dcg = dcg_at_k(y_true_sorted, k)
    idcg = dcg_at_k(sorted(y_true, reverse=True), k)
    return dcg / idcg if idcg > 0 else 0.0

def recall_at_k(y_true, y_score, k):
    top_k_items = np.argsort(y_score)[::-1][:k]
    hits = np.sum(y_true[top_k_items])
    return hits / np.sum(y_true)

# Function to evaluate the UserCF model
def evaluate_user_cf_model(model, test_data, k):
    ndcg_scores = []
    recall_scores = []

    # Get unique users
    unique_users = test_data['user'].unique()

    for user in unique_users:
        # Get items and labels for the user
        user_data = test_data[test_data['user'] == user]
        item_ids = user_data['item'].values
        labels = user_data['label'].values

        # Get scores for the items
        scores = model.predict(user=user, item=item_ids)

        # Calculate NDCG and Recall
        ndcg = ndcg_at_k(labels, scores, k)
        recall = recall_at_k(labels, scores, k)

        ndcg_scores.append(ndcg)
        recall_scores.append(recall)

    avg_ndcg = np.mean(ndcg_scores)
    avg_recall = np.mean(recall_scores)

    return {
        'NDCG@{}'.format(k): avg_ndcg,
        'Recall@{}'.format(k): avg_recall
    }

# Evaluate the model
eval_result = evaluate_user_cf_model(item_cf, test_ratings, k=5)
print(eval_result)

Detect 1 unknown interaction(s), position: [4]


C:\Users\Hooman\AppData\Local\Temp\ipykernel_17848\3108050799.py:17: RuntimeWarning: invalid value encountered in scalar divide
  return hits / np.sum(y_true)


Detect 1 unknown interaction(s), position: [95]
Detect 1 unknown interaction(s), position: [10]
Detect 1 unknown interaction(s), position: [60]
Detect 2 unknown interaction(s), position: [12, 54]
Detect 1 unknown interaction(s), position: [18]
Detect 1 unknown interaction(s), position: [51]
Detect 1 unknown interaction(s), position: [69]
Detect 1 unknown interaction(s), position: [2]
Detect 1 unknown interaction(s), position: [8]
Detect 1 unknown interaction(s), position: [123]
Detect 1 unknown interaction(s), position: [14]
Detect 1 unknown interaction(s), position: [38]
Detect 1 unknown interaction(s), position: [7]
Detect 1 unknown interaction(s), position: [0]
Detect 1 unknown interaction(s), position: [22]
Detect 1 unknown interaction(s), position: [19]
Detect 1 unknown interaction(s), position: [19]
Detect 8 unknown interaction(s), position: [129, 231, 15, 176, 146, 84, 182, 26]
Detect 1 unknown interaction(s), position: [75]
Detect 1 unknown interaction(s), position: [41]
Detect